# Full EDA and Modeling Notebook

Detailed exploratory analysis, visualizations, and training cells. Run cells sequentially.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline
repo_root = Path('.')
df = pd.read_csv('data/processed/cleaned_transactions.csv', parse_dates=['date'])
df.head()

In [ ]:
# Prepare monthly totals
monthly = df[df['is_expense']].groupby(df['date'].dt.to_period('M').astype(str)).agg(total_spent=('amount_abs','sum')).reset_index()
monthly['total_spent'] = monthly['total_spent'].astype(float)
monthly.head()

In [ ]:
# Plot monthly totals
plt.figure(figsize=(12,4))
plt.plot(monthly['year_month'] if 'year_month' in monthly.columns else monthly['date'].astype(str), monthly['total_spent'])
plt.title('Monthly total expenses')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Category spend
cat = df[df['is_expense']].groupby('category').agg(total=('amount_abs','sum')).reset_index().sort_values('total', ascending=False)
plt.figure(figsize=(10,6))
plt.bar(cat['category'].astype(str).head(20), cat['total'].head(20))
plt.xticks(rotation=45)
plt.title('Top categories by spending')
plt.tight_layout()
plt.show()

In [ ]:
# RandomForest baseline training example
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
monthly = monthly.sort_values('year_month' if 'year_month' in monthly.columns else 'date')
monthly['lag1'] = monthly['total_spent'].shift(1)
monthly['lag2'] = monthly['total_spent'].shift(2)
monthly['lag3'] = monthly['total_spent'].shift(3)
monthly = monthly.dropna().reset_index(drop=True)
monthly['target'] = monthly['total_spent'].shift(-1)
monthly = monthly.dropna().reset_index(drop=True)
X = monthly[['total_spent','lag1','lag2','lag3']].values
y = monthly['target'].values
if len(X) > 5:
    tscv = TimeSeriesSplit(n_splits=3)
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        m = RandomForestRegressor(n_estimators=100, random_state=42)
        m.fit(X_train, y_train)
        print('MAE:', mean_absolute_error(y_test, m.predict(X_test)))
else:
    print('Not enough data for time-series cross-validation')

## LSTM training (local)
The environment used to prepare this project did not have TensorFlow installed. To train an LSTM, run `python src/train_lstm.py` after installing TensorFlow (`pip install tensorflow`).